In [1]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [2]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
165,A widely unknown strange little western with m...,positive
765,I first saw Love in Limbo playing late on free...,positive
218,"I've gotta say, I usually like horror movies t...",negative
311,"Well, I was hoping I'd heard wrong about this ...",negative
676,The only redeeming feature of this movie is St...,negative


In [ ]:
# import nltk
# nltk.download('stopwords')
# nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aly98\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\aly98\AppData\Roaming\nltk_data...


True

In [6]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])  
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [7]:
df = normalize_text(df)
df.head()

,review,sentiment
165,widely unknown strange little western mindblow...,positive
765,first saw love limbo playing late free air tv ...,positive
218,gotta say usually like horror movie never seen...,negative
311,well hoping heard wrong film big fan ruggero d...,negative
676,redeeming feature movie steve carell like john...,negative


In [8]:
df['sentiment'].value_counts()

sentiment
negative    262
positive    238
Name: count, dtype: int64

In [14]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
165,widely unknown strange little western mindblow...,1
765,first saw love limbo playing late free air tv ...,1
218,gotta say usually like horror movie never seen...,0
311,well hoping heard wrong film big fan ruggero d...,0
676,redeeming feature movie steve carell like john...,0


In [15]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [16]:
vectorizer = CountVectorizer(max_features=100)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [18]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/alysahab/Mlops-Capstone-Project.mlflow')
dagshub.init(repo_owner='alysahab', repo_name='Mlops-Capstone-Project', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")


❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

Output()



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=12a37f35-5d67-4850-9e5c-1dfdcec57931&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=2a7fc97cc3a3f501b1900e2d35e2cef84f3842f1e7b60ffe23ab67e1c2027b5d




Accessing as alysahab

Initialized MLflow to track repo "alysahab/Mlops-Capstone-Project"

Repository alysahab/Mlops-Capstone-Project initialized!

2025/11/17 12:39:23 INFO mlflow.tracking.fluent: Experiment with name 'Logistic Regression Baseline' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/3e63b854f97a4f9c8bb353bc290012cb', creation_time=1763365163185, experiment_id='0', last_update_time=1763365163185, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [21]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 100)
        mlflow.log_param("test_size", 0.20)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2025-11-17 12:43:29,824 - INFO - Starting MLflow run...
2025-11-17 12:43:31,601 - INFO - Logging preprocessing parameters...
2025-11-17 12:43:33,147 - INFO - Initializing Logistic Regression model...
2025-11-17 12:43:33,147 - INFO - Fitting the model...
2025-11-17 12:43:33,161 - INFO - Model training complete.
2025-11-17 12:43:33,162 - INFO - Logging model parameters...
2025-11-17 12:43:33,714 - INFO - Making predictions...
2025-11-17 12:43:33,716 - INFO - Calculating evaluation metrics...
2025-11-17 12:43:33,748 - INFO - Logging evaluation metrics...
2025-11-17 12:43:36,113 - INFO - Saving and logging the model...
2025/11/17 12:43:41 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025-11-17 12:43:54,815 - INFO - Model training and logging completed in 23.21 seconds.
2025-11-17 12:43:54,818 - INFO - Accuracy: 0.6
2025-11-17 12:43:54,821 - INFO - Precision: 0.

🏃 View run beautiful-shoat-818 at: https://dagshub.com/alysahab/Mlops-Capstone-Project.mlflow/#/experiments/0/runs/adae594e3e2748d58e3f2550599e46f6
🧪 View experiment at: https://dagshub.com/alysahab/Mlops-Capstone-Project.mlflow/#/experiments/0
